Here is the solution for https://www.kaggle.com/c/titanic

In [16]:
import csv
import numpy as np

train_column_names = ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', \
                'Age', 'SibSp', 'Parch', 'Ticket','Fare', 'Cabin', 'Embarked'] 
test_column_names = ['PassengerId', 'Pclass', 'Name', 'Sex', \
                'Age', 'SibSp', 'Parch', 'Ticket','Fare', 'Cabin', 'Embarked'] 

# load data from csv file
def read_data(file, column_names):
    with open(file, 'r', errors='backslashreplace') as f:
        reader = csv.DictReader(c.replace('\0', '') for c in f)
        for row in reader:
            yield row

def load_data(file, column_names):
    data = [d for d in read_data(file, column_names)]
    return data

def print_data(data):
    for d in data:
        print(d)

def train(file, column_names):
    data = load_data(file, column_names)
    # print_data(data)
    
    

def test(file, column_names):
    pass

def output(accuracy, file):
    f = open(file, 'w')
    f.write('PassengerId,Survived')
    #for a in 
    f.close()

In [17]:
train_data_file = 'train.csv'
test_data_file = 'test.csv'
output_file = 'output.csv'
pipeline = train(train_data_file, train_column_names)
accuracy = test(test_data_file, train_column_names)
output(accuracy, output_file)